In [1]:
from datasets import load_dataset

import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

%load_ext autoreload
%autoreload 2

/home/colinhl/anaconda3/envs/sdd2025/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
ds = load_dataset("MKZuziak/ISIC_2019_224", split="train") # que le split train disponible

In [11]:
ds_small = ds.train_test_split(train_size=0.1, seed=42)["train"]

ds_split = ds_small.train_test_split(test_size=0.2, seed=42)

train_ds = ds_split["train"]
test_ds = ds_split["test"]

In [13]:
from utils import ISICDataset

my_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# On passe le dataset chargé précédemment à notre classe
custom_ds = ISICDataset(train_ds, transform=my_transform)

In [14]:
len(custom_ds)

2026

In [16]:
from utils import ISICDataset

my_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# On passe le dataset chargé précédemment à notre classe
custom_ds = ISICDataset(train_ds, transform=my_transform)

train_loader = DataLoader(
    custom_ds, 
    batch_size=16, 
    shuffle=True, # Important pour le déséquilibre
)

# --- ÉTAPE 3 : Affichage des 16 premières images ---
# On récupère un batch

batch = next(iter(train_loader))
images, labels = batch

# plt.figure(figsize=(8, 8))
# for i in range(16):
#     plt.subplot(4, 4, i + 1)
#     # PyTorch utilise (C, H, W), Matplotlib attend (H, W, C) -> on utilise permute
#     img_display = images[i].permute(1, 2, 0).numpy()
#     plt.imshow(img_display)
#     plt.title(f"Classe: {labels[i].item()}")
#     plt.axis("off")

# plt.tight_layout()
# plt.show()

In [17]:
from model import cnn_model
from train import setup_training, train_one_epoch

In [20]:
# Init the training
model = cnn_model(num_classes=8)
criterion, optimizer = setup_training(model, learning_rate=0.01, momentum=0.9)

DEVICE = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(DEVICE)
model = model.to(DEVICE)
# print(model)

train_losses = []
valid_losses = []

EPOCHS = 10
# loop over the dataset multiple times
for epoch in range(EPOCHS):
    model.train()
    train_epoch_loss = train_one_epoch(model, criterion, optimizer, train_loader, DEVICE)
    # model.eval()
    # valid_epoch_loss = valid_one_epoch(model, val_loader)

    print(f"EPOCH={epoch}, TRAIN={train_epoch_loss:.05f}")

    train_losses.append(train_epoch_loss)
    # valid_losses.append(valid_epoch_loss)

cpu


  0%|          | 0/127 [00:00<?, ?it/s]


KeyboardInterrupt: decoding with 'cp437' codec failed (KeyboardInterrupt: )

In [ ]:
from tqdm import tqdm

y_true = []
y_pred = []

# Send model to correct device
model.to(DEVICE)

# Put model in evaluatio mode (very important)
model.eval()

# Disable all gradients things
with torch.no_grad():
    for x, y_t in tqdm.tqdm(test_ds, "predicting"):
        x = x.reshape((-1,) + x.shape)
        x = x.to(DEVICE)
        y = model.forward(x)
        y = y.to("cpu").numpy()

        y_t = int(y_t.to("cpu").numpy())

        y_pred.append(y)
        y_true.append(y_t)


y_pred = np.concatenate(y_pred, axis=0)
y_true = np.asarray(y_true)

y_pred_classes = y_pred[:, 0] > 0.5